# MNEflow basic example

### This example assumes that basic example notebook was run and processed tfrecords were saved to 'path'.

In [1]:
import numpy as np
import mne
from mne.datasets import multimodal
import os
import tensorflow as tf

tf.get_logger().setLevel('ERROR')
tf.autograph.set_verbosity(0)
mne.set_log_level(verbose='CRITICAL')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import mneflow
print(mneflow.__version__)

0.5.4


### If we've already imported the data variables 'path' and 'data_id' to reuse the mne_sample_dataset. This won't work if trecords from the basic example were not saved.

In [2]:
import_opt = dict(path='C:\\data\\',
                  data_id='mne_sample_multimodal',
                  overwrite=False,
                  )

#here we use None instead of the first required argument
meta = mneflow.produce_tfrecords(None, **import_opt)

Metadata file found, restoring


### Alternatively, we can use path and data_id directly

In [3]:
path='C:\\data\\'
data_id='mne_sample_multimodal'
meta = mneflow.utils.load_meta(path + data_id)

### If metadata contains the trained model (i.e. the model was previously trained) the trained model can be restored using:

In [4]:
model = meta.restore_model()

Using class_subset with 7 classes:
Subset ratio 1.00, Multiplier 1.00
Using class_subset with 7 classes:
Subset ratio 1.00, Multiplier 1.00
Updating: meta.data
Setting reg for dmx, to l1
Built: dmx input: (None, 1, 301, 204)
Setting reg for tconv, to l1
Built: tconv input: (None, 1, 301, 32)
Setting reg for fc, to l1
Built: fc input: (None, 1, 61, 32)


### Restored models can continue training. 

In [5]:
model.train()

Updating: meta.train_params
Class weights:  None
Using class_subset with 7 classes:
Subset ratio 1.00, Multiplier 1.00
Epoch 1/10
8/8 - 2s - loss: 0.2812 - cat_ACC: 0.9974 - val_loss: 0.6246 - val_cat_ACC: 0.8899 - 2s/epoch - 217ms/step
Epoch 2/10
8/8 - 1s - loss: 0.2834 - cat_ACC: 0.9973 - val_loss: 0.6518 - val_cat_ACC: 0.8991 - 774ms/epoch - 97ms/step
Epoch 3/10
8/8 - 1s - loss: 0.2753 - cat_ACC: 0.9987 - val_loss: 0.6653 - val_cat_ACC: 0.8991 - 974ms/epoch - 122ms/step
Epoch 4/10
8/8 - 1s - loss: 0.2752 - cat_ACC: 0.9973 - val_loss: 0.6449 - val_cat_ACC: 0.9083 - 727ms/epoch - 91ms/step
Using class_subset with 7 classes:
Subset ratio 1.00, Multiplier 1.00
single_fold with 1 fold(s) completed. 

              Validation Performance: 
              Loss: 0.6246 +/- 0.0000.
              Metric: 0.8899 +/- 0.0000


              Test Performance: 
              Loss: 0.8452 +/- 0.0000.
              Metric: 0.8120 +/- 0.0000
Saving updated log to:  C:\data\models\lfcnn_log.csv


In [6]:
test_loss, test_acc = model.evaluate(meta.data['test_paths'])
print("Test set: Loss = {:.4f} Accuracy = {:.4f}".format(test_loss, test_acc))

Using class_subset with 7 classes:
Subset ratio 1.00, Multiplier 1.00
Test set: Loss = 0.8452 Accuracy = 0.8120


### Or predict new inputs

In [24]:
X, y = [row for row in model.dataset.val.take(1)][0]
y_pred = model.predict_sample(X[0])

In [28]:
print("Predicted: {}, Ground truth {}".format(y_pred[0], np.argmax(y[0])))

Predicted: 4, Ground truth 4
